## Executive Summary

This analysis reviewed the integrity of four datasets: `contracts.csv`, `merchants.csv`, `order_lines.csv`, and `orders.csv`, and calculated the attach rate for warranties overall and by merchant per month.

**Data Integrity Findings:** (Answer to Question 1, part 1)

*   **Missing Values:** Missing values were identified in `line_item_id` (contracts), `variant_id` and `is_warrantable` (order\_lines), and `shipping_country` (orders).

*   **Numerical Column Analysis:** Potential outliers were observed in `quantity` and `price` in the `order_lines` dataset. I'd suggest investigating these further.
*   **Categorical Column Consistency:** Inconsistent representations were found in `shipping_country` (e.g., 'United States' and 'US', 'United Kingdom' and 'UK'). and `source_name` (e.g., web, integers, draft orders, etc) in the `orders` dataset. Also, the `sourcekey` in the `merchants` table contains a format of `store_id` that is inconsistent with other store id's.

**Attach Rate Analysis:**

The overall unit attach rate was 3.57%. The attach rate varies significantly by merchant and across months. The opportunity analysis (see `opportunity_query`) highlighted the `total_warranty_sales` and `net_revenue` generated by each merchant.

**Business Development Recommendation:**

Based on the current attach rates and net revenue generated by merchants, the **Sports & Fitness Equipment** and **Home Security** industries are industries that I'd recommend for focused business development efforts. These industries contain multiple merchants with the highest net revenue, indicating potential for growth.

**Data Integrity Next Steps:** (Answer to Question 1, part 2)

*   Address missing values and standardize inconsistent categorical data for improved data quality.
*   Investigate factors contributing to higher attach rates in specific merchants and industries.
*   Utilize external market sizing data to further validate industry potential.
*   Develop targeted business development strategies for the identified high-potential industries and merchants.

In [73]:
import pandas as pd # for data wrangling in python
import sqlite3 # to show SQL knowledge


folder_path = '/content/drive/My Drive/Colab Projects/extend_july_2025/'
file_names = ['contracts.csv', 'merchants.csv', 'order_lines.csv', 'orders.csv']

# Load each CSV file into a pandas DataFrame
contracts_df = pd.read_csv(folder_path + file_names[0])
merchants_df = pd.read_csv(folder_path + file_names[1])
order_lines_df = pd.read_csv(folder_path + file_names[2])
orders_df = pd.read_csv(folder_path + file_names[3])

## Question 2 - Calculate attach rate overall and by merchant per month on a unit and dollar basis.

In [151]:

conn = sqlite3.connect(':memory:')


contracts_df.to_sql('contracts', conn, index=False)
order_lines_df.to_sql('order_lines', conn, index=False)
orders_df.to_sql('orders', conn, index=False)
merchants_df.to_sql('merchants', conn, index=False)

# Assumed attach rate defined as the percent of items purchased with a warranty over all the products purchased
# includes items with refunded warranty
attach_rate_overall_query = """
SELECT
    SUM(CASE WHEN C.contract_id IS NOT NULL THEN OL.quantity ELSE 0 END) AS total_purchased_items_with_warranty,
    SUM(CASE WHEN C.contract_id IS NULL AND OL.is_warranty = 0 THEN OL.quantity ELSE 0 END) AS total_purchased_items_without_warranty,
    ROUND(SUM(CASE WHEN C.contract_id IS NOT NULL THEN OL.quantity ELSE 0 END) * 100.0 / SUM(CASE WHEN C.contract_id IS NULL AND is_warranty = 0 THEN OL.quantity ELSE 0 END),2) AS overall_unit_attach_rate_percentage
FROM order_lines AS OL
LEFT JOIN contracts AS C
ON OL.line_item_id = C.line_item_id;
"""

# Caveats
# excludes stores without orders and not enabled merchants (e.g., RefurbPCLand)
attach_rate_by_merchant_query = """
SELECT
  M.name AS merchant_name,
  substr(O.ordered_at,1,7) AS order_month,
  ROUND(SUM(CASE WHEN C.contract_id IS NOT NULL THEN OL.quantity ELSE 0 END) * 100.0 / SUM(CASE WHEN C.contract_id IS NULL AND is_warranty = 0 THEN OL.quantity ELSE 0 END),2) AS unit_attach_rate_percent,
  ROUND(SUM(CASE WHEN C.contract_id IS NOT NULL THEN OL.product_purchase_price ELSE 0 END) * 100.0 / SUM(CASE WHEN C.contract_id IS NULL AND is_warranty = 0 THEN OL.product_purchase_price ELSE 0 END),2) AS dollar_attach_rate_percent
FROM order_lines AS OL
LEFT JOIN contracts AS C
  ON OL.line_item_id = C.line_item_id
    AND C.is_refunded = FALSE
INNER JOIN orders AS O
  ON OL.order_id = O.order_id
LEFT JOIN merchants AS M
  ON O.store_id = substr(M.sortkey,8)
WHERE
  M.approved = 1
GROUP BY
  M.name, 2
ORDER BY
  M.name, 2;
"""


opportunity_query = """
WITH MerchantAttachRate AS (
SELECT
  M.name AS merchant_name,
  ROUND(SUM(CASE WHEN C.contract_id IS NOT NULL THEN OL.quantity ELSE 0 END) * 100.0 / SUM(CASE WHEN C.contract_id IS NULL AND is_warranty = 0 THEN OL.quantity ELSE 0 END),2) AS unit_attach_rate_percent,
  ROUND(SUM(CASE WHEN C.contract_id IS NOT NULL THEN OL.product_purchase_price ELSE 0 END) * 100.0 / SUM(CASE WHEN C.contract_id IS NULL AND is_warranty = 0 THEN OL.product_purchase_price ELSE 0 END),2) AS dollar_attach_rate_percent
FROM order_lines AS OL
LEFT JOIN contracts AS C
  ON OL.line_item_id = C.line_item_id
    AND C.is_refunded = FALSE
INNER JOIN orders AS O
  ON OL.order_id = O.order_id
LEFT JOIN merchants AS M
  ON O.store_id = substr(M.sortkey,8)
WHERE
  M.approved = 1
GROUP BY
  M.name
ORDER BY
  M.name
), MerchantRevenue AS (
SELECT
  M.name AS merchant_name,
  SUM(C.plan_purchase_price * OL.quantity) AS total_warranty_sales,
  SUM(C.plan_purchase_price * OL.quantity) * (1- M.merchantcut) AS net_revenue
FROM order_lines AS OL
LEFT JOIN contracts AS C
  ON OL.line_item_id = C.line_item_id
LEFT JOIN merchants AS M
  ON C.store_id = substr(M.sortkey,8)
WHERE
    C.store_id IS NOT NULL
GROUP BY
  1
), OverallAttachRate AS (
  SELECT
    ROUND(SUM(CASE WHEN C.contract_id IS NOT NULL THEN OL.quantity ELSE 0 END) * 100.0 / SUM(CASE WHEN C.contract_id IS NULL AND is_warranty = 0 THEN OL.quantity ELSE 0 END),2) AS overall_unit_attach_rate_percentage
FROM order_lines AS OL
LEFT JOIN contracts AS C
ON OL.line_item_id = C.line_item_id
)
SELECT
  MR.merchant_name,
  MR.total_warranty_sales,
  MR.net_revenue,
  MAR.unit_attach_rate_percent,
  MAR.dollar_attach_rate_percent,
  (SELECT overall_unit_attach_rate_percentage FROM OverallAttachRate) AS overall_unit_attach_rate_percentage
FROM MerchantRevenue AS MR
INNER JOIN MerchantAttachRate AS MAR
  ON MR.merchant_name = MAR.merchant_name
ORDER BY
  MR.net_revenue DESC;
"""

attach_rate_overall = pd.read_sql_query(attach_rate_overall_query, conn)
attach_rate_by_merchant = pd.read_sql_query(attach_rate_by_merchant_query, conn)
warranty_sales = pd.read_sql_query(opportunity_query, conn)

print("Overall Attach Rate:")
display(attach_rate_overall)

print("Attach Rate by Merchant:")
display(attach_rate_by_merchant)

print("Opportunity:")
display(warranty_sales)

conn.close()

Overall Attach Rate:


,total_purchased_items_with_warranty,total_purchased_items_without_warranty,overall_unit_attach_rate_percentage
0,7415,207811,3.57


Attach Rate by Merchant:


,merchant_name,order_month,unit_attach_rate_percent,dollar_attach_rate_percent
0,CarStereoIsland,2020-03,17.12,7.43
1,CarStereoIsland,2020-04,14.31,6.86
2,CarStereoIsland,2020-05,11.98,5.83
3,DietTrackers,2020-04,0.67,1.29
4,DietTrackers,2020-05,1.83,6.55
5,DroneWarehouse,2020-04,0.00,0.00
6,DroneWarehouse,2020-05,0.83,2.12
7,ElectricSkatePark,2020-03,7.46,7.62
8,ElectricSkatePark,2020-04,10.11,9.47
9,ElectricSkatePark,2020-05,14.58,15.67


Opportunity:


,merchant_name,total_warranty_sales,net_revenue,unit_attach_rate_percent,dollar_attach_rate_percent,overall_unit_attach_rate_percentage
0,SkateboardsUSA,40911.71,28638.1970,1.81,9.72,3.57
1,FitnessWareables,32618.11,22832.6770,9.48,12.01,3.57
2,HomeSecurityMart,19968.48,14976.3600,6.37,7.79,3.57
3,LuxuryWatchDepot,22418.00,14571.7000,1.45,1.47,3.57
4,Photos4Grandma,20708.09,14495.6630,5.26,5.11,3.57
5,HVAC4U,20533.50,13346.7750,9.13,9.13,3.57
6,PowerMassagerPros,16854.93,12641.1975,5.53,7.13,3.57
7,SecurityCamDirect,14154.59,10615.9425,30.67,22.54,3.57
8,CarStereoIsland,9982.48,7486.8600,13.93,6.53,3.57
9,DietTrackers,5036.60,3525.6200,1.50,5.04,3.57


## Question 4. Next Steps - Further Work

### Question A: What makes people buy warranties for different products and from different merchants?

This is a big one because if we can figure out why some customers add warranties and others don't, we can strategically increase our sales across the board, which directly impacts our net revenue. What's caught my eye in the current data is just how much the attach rates—the percentage of products sold with a warranty—vary across different merchants and over time. This tells me there's definitely something influencing customer decisions when it comes to warranties.

To really get to the bottom of this, I'd love to see some more data. Things like specific product details (think product type, price, brand, and even the nitty-gritty of the warranty coverage), customer demographics or behavior (if we have it, knowing more about our customers and their past purchases could reveal patterns), and details about how warranties are presented on our website or sales platforms (where are they placed? What's the customer support like?). Plus, any data on marketing or promotions related to warranties would be super helpful.

Once we have all that, we can combine it with our existing data and start looking for connections. My goal would be to build a predictive model that helps us pinpoint the most significant factors driving warranty purchases. With those insights, we can then create targeted strategies—maybe optimizing how we present warranties, tailoring offers to specific customer groups, or focusing on products where we see the most potential.

### Question B: Why are some contracts being refunded, and how does that vary by merchant, plan, or product?
High refund rates are a red flag because they directly hit our net revenue. They can also signal problems with product quality, warranty terms, or even just customers not fully understanding what they're buying. If we can identify and fix the root causes, we can improve both our profitability and customer happiness. The `is_refunded` column in our `contracts_df` definitely caught my interest here; it suggests there are patterns in refund behavior that we should investigate.

To understand this better, I'd want to know the specific reason for each refund (was the product returned? Was the customer unhappy? Did they simply not need the warranty?). Any data from customer service interactions about warranty or product issues would also be incredibly valuable, as would information on product returns that might be linked to warranty refunds.

With this additional data, we can analyze refund rates by merchant, plan, product, and the specific reason for the refund. This will help us spot any major outliers or trends. From there, we can investigate the most common reasons for refunds and work with the relevant teams—like product development, customer service, or our merchant partners—to tackle those underlying issues head-on.